In [ ]:
# numpy 로드하여 np로 사용
# pandas 로드하여 pd로 사용
# matplotlib.pyplot 로드하여 plt로 사용
# seaborn 로드하여 sns로 사용
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

In [ ]:
import pandas as pd

# 상대경로 기준: check.ipynb → code/visualization/ → ../ → ../ → data/
df = pd.read_csv("../../data/Mcdonalds_data.csv")

# 데이터 확인
print(df.head())
print(df.columns)

### 스크랩핑을 위한 모듈 다운 및 확인

In [ ]:
%pip show selenium webdriver-manager

In [ ]:
%pip show selenium

### Url 별 카테고리, 메뉴, 가격 스크랩핑
- 추후 아침메뉴 스크랩 코드 추가 작성 필요

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import csv
import os
import time

# ✅ 카테고리별 URL 딕셔너리
category_urls = {
    "버거 & 세트": "https://www.mcdelivery.co.kr/kr/browse/menu.html?daypartId=1&catId=11",
    "스낵 & 사이드": "https://www.mcdelivery.co.kr/kr/browse/menu.html?daypartId=1&catId=13",
    "음료": "https://www.mcdelivery.co.kr/kr/browse/menu.html?daypartId=1&catId=14",
    "디저트": "https://www.mcdelivery.co.kr/kr/browse/menu.html?daypartId=1&catId=15",
    "해피밀": "https://www.mcdelivery.co.kr/kr/browse/menu.html?daypartId=1&catId=16"
}

# ✅ 셀레니움 브라우저 설정
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

menu_data = []

# ✅ 각 URL 순회하며 데이터 수집
for category_name, url in category_urls.items():
    driver.get(url)
    time.sleep(3)

    products = driver.find_elements(By.CSS_SELECTOR, ".product-card.product-card--standard")

    for product in products:
        try:
            name = product.find_element(By.CLASS_NAME, "product-title").text.strip()
            price = product.find_element(By.CLASS_NAME, "starting-price").text.strip()
            price = price.replace("₩", "").replace(",", "")
            menu_data.append((category_name, name, int(price)))
        except:
            continue

driver.quit()

# ✅ 저장 디렉토리 생성
output_dir = os.path.join(os.getcwd(), "output")
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, "Mcdelivery_menu_prices.csv")

# ✅ CSV 저장
with open(output_file, "w", encoding="utf-8-sig", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["카테고리", "메뉴", "가격"])
    writer.writerows(menu_data)

print(f"✅ 저장 완료! 경로: {output_file}")